In [1]:
import { Langfuse } from "https://esm.sh/langfuse"

In [2]:
import { config } from "https://deno.land/x/dotenv/mod.ts";
import {z} from 'https://deno.land/x/zod@v3.23.8/mod.ts';
import { StructuredTool } from "https://esm.sh/@langchain/core/tools";
import { formatToOpenAITool } from "https://esm.sh/@langchain/openai";
import OpenAI from "https://deno.land/x/openai@v4.51.0/mod.ts";

const env = await config(); 

In [3]:
const LANGFUSE_SECRET_KEY = env.LANGFUSE_SECRET_KEY

In [4]:
const langfuse = new Langfuse({
    release: "v1.0.0", 
    requestTimeout: 10000,
    secretKey: env.LANGFUSE_SECRET_KEY,
    publicKey: env.LANGFUSE_PUBLIC_KEY,
    baseUrl: "http://localhost:3001",
});

In [5]:
const trace = langfuse.trace({
  name: "catch-all",
  metadata: { user: "peter@peter.com" },
  tags: ["dev"],
});

In [6]:
let messages = 'Please add 500 and 465.'

In [7]:
const generation = trace.generation({
  name: "chat-completion",
  model: "gpt-3.5-turbo-0125",
  modelParameters: {
    temperature: 0.7,
    maxTokens: 2000,
  },
  input: messages,
});

In [8]:
const OPENAI_API_KEY = env.OPENAI_API_KEY

In [9]:
import { ChatOpenAI } from "https://esm.sh/@langchain/openai";

In [10]:
const openai_client = new OpenAI({apiKey:OPENAI_API_KEY})

In [11]:
const client = new ChatOpenAI({apiKey:OPENAI_API_KEY})

In [12]:
await client.invoke("hi there")

g {
  lc_serializable: true,
  lc_kwargs: {
    content: "Hello! How can I assist you today?",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Hello! How can I assist you today?",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 9, promptTokens: 9, totalTokens: 18 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: [],
  usage_metadata: { input_tokens: 9, output_tokens: 9, total_tokens: 18 }
}

In [13]:
class AddNums extends StructuredTool {
    name = "add_numbers";
    description = "add two numbers together";

    schema =  z.object({
        num_1: z.number().describe("the first number to add"),
        num_2: z.number().describe("the second number to add"),
    })

    constructor() {
        super();
    }

    _call(input: z.infer<typeof this['schema']>): Promise<string> {
        return Promise.resolve(JSON.stringify(input, null, 2));
    } 
}

In [14]:
const addNums = new AddNums()

In [15]:
const tool = new AddNums()

In [16]:
tool._call({num_1: 3, num_2: 4}).then(console.log)

{
  "num_1": 3,
  "num_2": 4
}


Promise { undefined }

In [17]:
tool.schema.describe()

ZodObject {
  _type: undefined,
  _output: undefined,
  _input: undefined,
  _def: {
    shape: [Function: shape],
    unknownKeys: "strip",
    catchall: ZodNever {
      _type: undefined,
      _output: undefined,
      _input: undefined,
      _def: { typeName: "ZodNever" },
      spa: [Function: bound safeParseAsync] AsyncFunction,
      parse: [Function: bound parse],
      safeParse: [Function: bound safeParse],
      parseAsync: [Function: bound parseAsync] AsyncFunction,
      safeParseAsync: [Function: bound safeParseAsync] AsyncFunction,
      refine: [Function: bound refine],
      refinement: [Function: bound refinement],
      superRefine: [Function: bound superRefine],
      optional: [Function: bound optional],
      nullable: [Function: bound nullable],
      nullish: [Function: bound nullish],
      array: [Function: bound array],
      promise: [Function: bound promise],
      or: [Function: bound or],
      and: [Function: bound and],
      transform: [Function: boun

In [18]:
typeof(3)

"number"

In [19]:
addNums

AddNums {
  lc_serializable: false,
  lc_kwargs: {},
  lc_runnable: true,
  name: "add_numbers",
  verbose: false,
  callbacks: undefined,
  tags: [],
  metadata: {},
  returnDirect: false,
  description: "add two numbers together",
  schema: ZodObject {
    _type: undefined,
    _output: undefined,
    _input: undefined,
    _def: {
      shape: [Function: shape],
      unknownKeys: "strip",
      catchall: ZodNever {
        _type: undefined,
        _output: undefined,
        _input: undefined,
        _def: { typeName: "ZodNever" },
        spa: [Function: bound safeParseAsync] AsyncFunction,
        parse: [Function: bound parse],
        safeParse: [Function: bound safeParse],
        parseAsync: [Function: bound parseAsync] AsyncFunction,
        safeParseAsync: [Function: bound safeParseAsync] AsyncFunction,
        refine: [Function: bound refine],
        refinement: [Function: bound refinement],
        superRefine: [Function: bound superRefine],
        optional: [Function

In [20]:
const addTool = formatToOpenAITool(new AddNums())

In [21]:
addTool

{
  type: "function",
  function: {
    name: "add_numbers",
    description: "add two numbers together",
    parameters: {
      type: "object",
      properties: {
        num_1: { type: "number", description: "the first number to add" },
        num_2: { type: "number", description: "the second number to add" }
      },
      required: [ "num_1", "num_2" ],
      additionalProperties: false,
      "$schema": "http://json-schema.org/draft-07/schema#"
    }
  }
}

In [22]:
await openai_client.chat.completions.create({
      messages: [{ role: 'user', content: "hello tell me a joke" }],
      model: 'gpt-3.5-turbo-0125',
    });

{
  id: "chatcmpl-9bpvteTN8HsNMd7IvKByROzaJ4sO0",
  object: "chat.completion",
  created: 1718804793,
  model: "gpt-3.5-turbo-0125",
  choices: [
    {
      index: 0,
      message: {
        role: "assistant",
        content: "Sure! Here's a joke for you:\n" +
          "\n" +
          "Why couldn't the bicycle stand up by itself?\n" +
          "Because it was two-tired!"
      },
      logprobs: null,
      finish_reason: "stop"
    }
  ],
  usage: { prompt_tokens: 12, completion_tokens: 26, total_tokens: 38 },
  system_fingerprint: null
}

In [23]:
addTool

{
  type: "function",
  function: {
    name: "add_numbers",
    description: "add two numbers together",
    parameters: {
      type: "object",
      properties: {
        num_1: { type: "number", description: "the first number to add" },
        num_2: { type: "number", description: "the second number to add" }
      },
      required: [ "num_1", "num_2" ],
      additionalProperties: false,
      "$schema": "http://json-schema.org/draft-07/schema#"
    }
  }
}

In [24]:
let tools = [addTool]

In [25]:
tools[0]

{
  type: "function",
  function: {
    name: "add_numbers",
    description: "add two numbers together",
    parameters: {
      type: "object",
      properties: {
        num_1: { type: "number", description: "the first number to add" },
        num_2: { type: "number", description: "the second number to add" }
      },
      required: [ "num_1", "num_2" ],
      additionalProperties: false,
      "$schema": "http://json-schema.org/draft-07/schema#"
    }
  }
}

In [26]:
let res = await openai_client.chat.completions.create({
    model: 'gpt-3.5-turbo-0125',
    messages: [{ role: 'user', content: messages }],
    tools: tools
    });

In [27]:
let responseMessage = res.choices[0].message

In [29]:
generation.end({
  output: responseMessage,
});;

A {
  client: d {
    debugMode: false,
    pendingPromises: { "5e83c738-708a-4231-88a4-7e486a6c76ce": Promise { <pending> } },
    _events: S { events: {} },
    enabled: true,
    publicKey: "pk-lf-409b1062-497b-47b7-b167-89d267ea9469",
    secretKey: "sk-lf-926cd2e9-b209-4667-bbdf-2118a9247d34",
    baseUrl: "http://localhost:3001",
    flushAt: 15,
    flushInterval: 10000,
    release: "v1.0.0",
    _retryOptions: { retryCount: 3, retryDelay: 3000, retryCheck: [Function: q] },
    requestTimeout: 10000,
    sdkIntegration: "DEFAULT",
    _promptCache: T { _cache: Map(0) {}, _defaultTtlSeconds: 60 },
    _storageKey: "lf_pk-lf-409b1062-497b-47b7-b167-89d267ea9469_langfuse",
    _storage: {
      getItem: [Function: getItem],
      setItem: [Function: setItem],
      removeItem: [Function: removeItem],
      clear: [Function: clear],
      getAllKeys: [Function: getAllKeys]
    },
    _storageCache: { queue: [] },
    _flushTimer: null
  },
  id: "c0d4d41c-1b7e-495b-aac5-91c5085c94e

In [30]:
responseMessage.tool_calls

[
  {
    id: "call_q79ZMNDBzAYtp52jwyZ5fjv1",
    type: "function",
    function: { name: "add_numbers", arguments: '{"num_1":500,"num_2":465}' }
  }
]

In [31]:
let params
if (responseMessage.tool_calls) {
    params = JSON.parse(responseMessage.tool_calls[0].function.arguments);
    console.log(params)
}

{ num_1: 500, num_2: 465 }


In [32]:
params

{ num_1: 500, num_2: 465 }

In [33]:
function addingMachine ({num_1, num_2}) {
    return num_1 + num_2
}

In [34]:
console.log(addingMachine(params))

965
